In [1]:
from scipy.sparse import csr_matrix,lil_matrix
import scipy as sp
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import vstack,hstack
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
from scipy.special import logit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, accuracy_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

In [2]:
data = pd.read_csv('dataset_latest.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102192 entries, 0 to 102191
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Unnamed: 0   102192 non-null  int64 
 1   source_id    102192 non-null  int64 
 2   source       102192 non-null  object
 3   target_id    102192 non-null  int64 
 4   target       102192 non-null  object
 5   weight       102192 non-null  int64 
 6   s_node_id    102192 non-null  int64 
 7   s_group      102192 non-null  object
 8   s_timestamp  102192 non-null  int64 
 9   t_node_id    102192 non-null  int64 
 10  t_group      102192 non-null  object
 11  t_timestamp  102192 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 9.4+ MB


In [3]:
nor_weight=data.groupby(['s_timestamp', 's_group'])['weight'].max()
nor_weight

s_timestamp  s_group   
0            Al-Shabaab     3
             Boko Haram     9
             Taliban       14
1            Al-Shabaab     2
             Boko Haram     2
                           ..
312          Taliban       52
313          Al-Shabaab     3
             Boko Haram     6
             ISIS           5
             Taliban       59
Name: weight, Length: 1214, dtype: int64

In [4]:
def norm_weight(df, normdf):
    x=df.values
    norm =[]
    for a in range(x.shape[0]):
        denom = normdf[df['s_timestamp'][a]][df['s_group'][a]]
        norm.append(x[a,5]/denom)
    return pd.Series(norm)
        

In [5]:
data['norm'] = norm_weight(data, nor_weight)
data.head(5)

,Unnamed: 0,source_id,source,target_id,target,weight,s_node_id,s_group,s_timestamp,t_node_id,t_group,t_timestamp,norm
0,0,1529,Bombing/Explosion,1529,Bombing/Explosion,3,1529,Al-Shabaab,0,1529,Al-Shabaab,0,1.000000
1,1,274,Armed Assault,5071,Firearms,3,274,Boko Haram,0,5071,Boko Haram,0,0.333333
2,2,274,Armed Assault,6185,Government (General),1,274,Boko Haram,0,6185,Boko Haram,0,0.111111
3,3,274,Armed Assault,7498,Incendiary,3,274,Boko Haram,0,7498,Boko Haram,0,0.333333
4,4,274,Armed Assault,9088,Police,2,274,Boko Haram,0,9088,Boko Haram,0,0.222222


In [6]:
y=pd.get_dummies(data.s_group)
data.join(y)

,Unnamed: 0,source_id,source,target_id,target,weight,s_node_id,s_group,s_timestamp,t_node_id,t_group,t_timestamp,norm,Al-Shabaab,Boko Haram,ISIS,Taliban
0,0,1529,Bombing/Explosion,1529,Bombing/Explosion,3,1529,Al-Shabaab,0,1529,Al-Shabaab,0,1.000000,1,0,0,0
1,1,274,Armed Assault,5071,Firearms,3,274,Boko Haram,0,5071,Boko Haram,0,0.333333,0,1,0,0
2,2,274,Armed Assault,6185,Government (General),1,274,Boko Haram,0,6185,Boko Haram,0,0.111111,0,1,0,0
3,3,274,Armed Assault,7498,Incendiary,3,274,Boko Haram,0,7498,Boko Haram,0,0.333333,0,1,0,0
4,4,274,Armed Assault,9088,Police,2,274,Boko Haram,0,9088,Boko Haram,0,0.222222,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102187,102187,10557,Private Citizens & Property,5571,Firearms,2,10557,ISIS,313,5571,ISIS,313,0.400000,0,0,1,0
102188,102188,10557,Private Citizens & Property,7196,Hostage Taking (Kidnapping),1,10557,ISIS,313,7196,ISIS,313,0.200000,0,0,1,0
102189,102189,10557,Private Citizens & Property,7731,Incendiary,1,10557,ISIS,313,7731,ISIS,313,0.200000,0,0,1,0
102190,102190,10557,Private Citizens & Property,742,Armed Assault,2,10557,ISIS,313,742,ISIS,313,0.400000,0,0,1,0


In [7]:
import pandas as pd 
class loader:
    
    def __init__(self):
        self.countID=0
        self.G={}
        self.co={}
        self.revco={}
        self.y=[]
        self.l=[]
        self.t=[]
    
    def nodeID(self,x, label_g, label_a, t):
        if x not in self.co:
            self.co[x]=self.countID
            self.countID=self.countID+1
            self.revco[self.co[x]]=x
            self.y.append(label_g)
            self.l.append([label_a])
            self.t.append(t)
        return self.co[x]
    
    def read(self,df):
        x=df.values
        for a in range(x.shape[0]):
            i=self.nodeID(x[a,1], x[a,7], x[a,2], x[a,8])
            j=self.nodeID(x[a,3], x[a,10], x[a,4], x[a,11])
            w=(x[a,12])
            self.addEdge((i,j), w)
        #self.fixG()
        
        
    
    def storeEmb(self,file,data):
        file1 = open(file, 'w') 
        for a in range(data.shape[0]):
            s=''+str(int(self.revco[a]))
            for b in range(data.shape[1]):
                s+=' '+str(data[a,b])
            file1.write(s+"\n")
        file1.close()
            
    
    def fixG(self):
        for g in range(len(self.G)):
            self.G[g]=np.array([x for x in self.G[g]])

    def addEdge(self,s, w):
        (l1,l2)=s
        if l1 not in self.G:
            self.G[l1]={}
        if l2 not in self.G:
            self.G[l2]={}
        self.G[l1][l2]=w
        self.G[l2][l1]=w
        
    
        

In [8]:
from scipy.sparse import lil_matrix,csc_matrix
class node:
    def __init__(self):
        self.id=-1
        self.dictionary={}

class mappa:
    def __init__(self):
        self.node=node()
        self.count=0
    
    def get(self,X):
        (_,b)=X.nonzero()
        c=self.node
        for x in b:
            if x not in c.dictionary:
                c.dictionary[x]=node()
            c=c.dictionary[x]
            f=int(X[0,x])
            if f not in c.dictionary:
                c.dictionary[f]=node()
            c=c.dictionary[f]
        if c.id==-1:
            c.id=self.count
            self.count=self.count+1
            return (c.id,1)
        return (c.id,0)
    
    def load(self,X):
        l=[]
        k=[]
        for x in range(X.shape[0]):
            (a,b)=self.get(X[x,:])
            l.append(a)
            if b==1:
                k.append(x)
        return l,k

def fast(G,l,iter=100):
    res=[]
    nv=len(G) 
    size=0
    emb=csc_matrix((nv, 1), dtype=np.int8)
    for i in range(iter):
        print(i)
        print(emb.shape)
        print(l.shape)
        ma=mappa()
        ll,_=ma.load(lil_matrix(hstack([emb,lil_matrix(l)])))
        emb=lil_matrix((nv, ma.count))
        for i in range(nv):
            for y in G[i]:
                j=ll[y]
                emb[i,j]+=G[i][y]
        if emb.shape[1]!=1 and emb.shape[1]==size:
            break
        else:
            size=emb.shape[1]
            res.append(emb)
        #current=hstack(res)
    print(iter)
    print(emb.shape)
    return hstack(res)
    #return emb

In [9]:
l=loader()
l.read(data)

In [10]:
l.l=np.array(l.l)

In [11]:
l.l=OneHotEncoder().fit_transform(l.l)

In [12]:
emb=fast(l.G,l.l,iter=100)
for i in [400]:
    svd = TruncatedSVD(n_components=min(i,emb.shape[1]-1),n_iter=10,random_state=1)
    X=svd.fit_transform(emb)

0
(12574, 1)
(12574, 37)
1
(12574, 37)
(12574, 37)
2
(12574, 9641)
(12574, 37)
3
(12574, 12407)
(12574, 37)
4
(12574, 12419)
(12574, 37)
100
(12574, 12419)


In [13]:
X.shape

(12574, 400)

In [14]:
train = X[0: 9427]
train = train.copy()
train_labels = l.y[0 : 9427]
train_labels = train_labels.copy()
test = X[9427 :]
test = test.copy()
test_labels = l.y[9427 :]
test_labels = test_labels.copy()

In [15]:
clf = ExtraTreesClassifier(n_estimators=400)
clf.fit(train, train_labels)
pred =clf.predict(test)

In [16]:
conf_mat = confusion_matrix(test_labels, pred)
conf_mat

array([[429, 121,  19, 159],
       [101, 367,  72,  32],
       [ 92,  74, 300, 358],
       [ 85,  51, 164, 723]])

In [17]:
print(classification_report(test_labels, pred))

              precision    recall  f1-score   support

  Al-Shabaab       0.61      0.59      0.60       728
  Boko Haram       0.60      0.64      0.62       572
        ISIS       0.54      0.36      0.44       824
     Taliban       0.57      0.71      0.63      1023

    accuracy                           0.58      3147
   macro avg       0.58      0.58      0.57      3147
weighted avg       0.58      0.58      0.57      3147

